In [23]:
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
# import h5py
np.random.seed(1337) # for reproducibility

from keras.preprocessing import sequence
from keras.optimizers import RMSprop
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution1D,Convolution2D, MaxPooling1D
from keras.regularizers import l2, activity_l1
from keras.constraints import maxnorm
from keras.layers.recurrent import LSTM, GRU
from keras.callbacks import ModelCheckpoint, EarlyStopping

from keras.optimizers import Adam, SGD

from keras import initializations

# In[14]:

def my_init(shape, name=None):
    return initializations.normal(shape, scale=0.001, name=name)



In [24]:
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
# import h5py
np.random.seed(1337) # for reproducibility

from keras.preprocessing import sequence
from keras.optimizers import RMSprop
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution1D,Convolution2D, MaxPooling1D
from keras.regularizers import l2, activity_l1
from keras.constraints import maxnorm
from keras.layers.recurrent import LSTM, GRU
from keras.callbacks import ModelCheckpoint, EarlyStopping

from keras.optimizers import Adam, SGD

from keras import initializations

# In[14]:

def my_init(shape, name=None):
    return initializations.normal(shape, scale=0.001, name=name)


#Load data directly from file
f = open('goodmuscontinuous.txt', 'r')
y = []
for line in f:
	y.append(float(line))

f = open('goodseqsBinary.txt', 'r')
seqs = []
for line in f:
	seqs.append(line.strip('\n'))

# ATGC to one hot encoding
seq_map = {'A': np.array([1,0,0,0]), 'T': np.array([0,1,0,0]), 'G': np.array([0,0,1,0]), 'C': np.array([0,0,0,1])}

#sequences to matrix form
X = []
for seq in seqs:
	X.append(np.vstack([seq_map[s] for s in seq]))

# zero pad sequences (end)
max_len = 0
for s in X:
	if len(s)>max_len:
		max_len = len(s)
for i in range(len(X)):
	p = max_len - X[i].shape[0]
	if p>0:
		X[i] = np.vstack([X[i], np.zeros((p,4))])

X= np.asarray(X)
y = np.asarray(y)
print X.shape


(8536, 111, 4)


In [25]:
print X[1870]

[[ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]
 [ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  1.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  1.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  1.  0

In [26]:
#shuffle indices...not data!
idx = range(len(X))
np.random.shuffle(idx)
train_idx = idx[:int(len(X)*.8)]
valid_idx = idx[int(len(X)*.8):int(len(X)*.9)]

#split into train/valid
X_train = X[train_idx]
X_valid = X[int(len(X)*.8):int(len(X)*.9)]
X_test = X[int(len(X)*.9):]

y_train = y[:int(len(X)*.8)]
y_valid = y[int(len(X)*.8):int(len(X)*.9)]
y_test = y[int(len(X)*.9):]

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[2], 1, X_train.shape[1])) #(N, F, H, W)
X_valid = np.reshape(X_valid, (X_valid.shape[0], X_valid.shape[2], 1, X_valid.shape[1])) #(N, F, H, W)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[2], 1, X_test.shape[1])) #(N, F, H, W)

print X_train.shape


(6828, 4, 1, 111)


In [27]:
lr = 1e-4#learning rate
reg = 1e-2

print 'building model'
model = Sequential()
model.add(Convolution2D(64, 4, 25, border_mode='same', input_shape=(X_train.shape[1], 1, X_train.shape[3]), W_regularizer=l2(reg), init=my_init))
model.add(Activation('tanh'))
model.add(Convolution2D(128, 4, 25, border_mode='same', W_regularizer=l2(reg), init=my_init))
model.add(Activation('tanh'))
model.add(Convolution2D(256, 4, 25, border_mode='same', W_regularizer=l2(reg), init=my_init))
model.add(Activation('tanh'))
model.add(Flatten())

model.add(Dense(1000))
model.add(Dense(1))
model.add(Activation('tanh')) #[-1, 1]


# In[17]:
adam = Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0005)

# sgd = SGD(lr=lr, momentum=0.9)
model.compile(loss='mse',
              optimizer='adam', metrics=['mse'])


print 'compiling model'

building model
compiling model


In [28]:
earlystopper = EarlyStopping(monitor='val_loss', patience=2, verbose=1)
model.fit(X_train, y_train,nb_epoch=3, batch_size=512, verbose=1, show_accuracy=True, validation_data=(X_valid, y_valid),callbacks=[earlystopper])



Train on 6828 samples, validate on 854 samples
Epoch 1/3
6828/6828 [==============================] - 168s - loss: 1.7007 - mean_squared_error: 1.6257 - val_loss: 2.2082 - val_mean_squared_error: 2.2082
Epoch 2/3
6828/6828 [==============================] - 181s - loss: 1.6043 - mean_squared_error: 1.4766 - val_loss: 2.2082 - val_mean_squared_error: 2.2082
Epoch 3/3
6828/6828 [==============================] - 179s - loss: 1.5593 - mean_squared_error: 1.4766 - val_loss: 2.2082 - val_mean_squared_error: 2.2082


In [15]:
%matplotlib 

out=model.predict(X_test, batch_size=512, verbose=1)
plt.plot(y_test,out,'ro')

model.summary()


Using matplotlib backend: MacOSX
2846/2846 [==============================] - 24s    
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_7 (Convolution2D)  (None, 64, 1, 113)    25664       convolution2d_input_3[0][0]      
____________________________________________________________________________________________________
activation_9 (Activation)        (None, 64, 1, 113)    0           convolution2d_7[0][0]            
____________________________________________________________________________________________________
convolution2d_8 (Convolution2D)  (None, 128, 1, 113)   819328      activation_9[0][0]               
____________________________________________________________________________________________________
activation_10 (Activation)       (None, 128, 1, 113)   0           convolution2d_8[0][0]            
_____